In [1]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# y_train.shape is 2d, (50000, 1). While Keras is smart enough to handle this
# it's a good idea to flatten the array.

y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

Using TensorFlow backend.


170500096/170498071 [==============================] - 42s 0us/step


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify = y_train)


# Define model pipeline

In [ ]:
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    iaa.OneOf([
        iaa.Crop(px=(0, 3)), # crop images from each side by 0 to 3px (randomly chosen)
        # it doesn't make sense to flip some of the signs
        #iaa.Fliplr(0.5), # horizontally flip 50% of the images
        iaa.GaussianBlur(sigma=(0, 1.0)), # blur images with a sigma of 0 to 3.0
        iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
        iaa.PerspectiveTransform(scale=(0.1, 0.2)) 
    ])
])

X_train_aug = seq.augment_images(X_train)

In [ ]:
def rgb2y(array):
    
    array_y = np.empty((len(array), 32, 32, 1))
    
    for i in range(len(array)):
    
        img_yuv = cv2.cvtColor(np.array(array[i], dtype=np.uint8), cv2.COLOR_BGR2YUV, 1)
        y, u, v = cv2.split(img_yuv)
        
        y_reshaped = np.reshape(y, y.shape + (1,))

        array_y[i] = y_reshaped
        
    return array_y

In [ ]:
def global_contrast_normalization(array, s=1, lmda=10, epsilon=0.000000001):    
    
    # https://datascience.stackexchange.com/questions/15110/how-to-implement-global-contrast-normalization-in-python

    array_y = np.empty((len(array), 32, 32, 1))
    
    for i in range(len(array)):
    
        X_average = np.mean(array[i])
    
        X = array[i] - X_average

        contrast = np.sqrt(lmda + np.mean(X**2))

        X = s * X / max(contrast, epsilon)

        array_y[i] = X
        
    return array_y

In [ ]:
X_train.shape